# Experiment 035G: Phase Relationship Detection

**AKIRA Project - Oscar Goldman - Shogu Research Group @ Datamutant.ai**

---

## Goal

Test whether AQ have detectable phase structure, not just magnitude clustering.

From `ACTION_QUANTA.md`:
```
AQ have four properties: magnitude, phase, frequency, coherence
Phase enables bonding via alignment
```

This experiment addresses:
1. **Phase extraction**: Can we detect phase structure in activation patterns?
2. **Phase alignment**: Do components of bonded states show aligned phases?
3. **Phase interference**: Do incompatible AQ show phase opposition?

---

## Hypothesis

If AQ phase is real:
1. Components of bonded states should have ALIGNED phases (low circular variance)
2. Random AQ combinations should have HIGH phase variance
3. Incompatible AQ (e.g., FLEE + APPROACH) should show phase opposition

---

## 1. Setup

In [ ]:
# Install dependencies (uncomment for Colab)
# !pip install transformers torch numpy scikit-learn matplotlib seaborn scipy -q

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
from scipy.signal import hilbert
from scipy import stats
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple, Optional, Any
from dataclasses import dataclass, field
import warnings
import json
from tqdm import tqdm
import gc

warnings.filterwarnings('ignore')

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {DEVICE}")
print(f"PyTorch version: {torch.__version__}")
if DEVICE == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 2. Configuration

In [ ]:
@dataclass
class ExperimentConfig:
    """Configuration for phase relationship experiment."""
    
    model_name: str = "gpt2-medium"
    model_path: str = "gpt2-medium"
    
    # Samples per category
    samples_per_category: int = 50
    
    # Layers to analyze
    layers_to_probe: List[int] = field(default_factory=lambda: [0, 4, 8, 12, 16, 20, 23])
    
    # Statistical parameters
    random_seed: int = 42
    
    def __post_init__(self) -> None:
        np.random.seed(self.random_seed)
        torch.manual_seed(self.random_seed)


config = ExperimentConfig()
print(f"Model: {config.model_name}")
print(f"Layers to probe: {config.layers_to_probe}")
print(f"Samples per category: {config.samples_per_category}")

## 3. Circular Statistics Functions

Phase is circular (wraps around at 2*pi), so we need circular statistics.

In [ ]:
def circular_mean(phases: np.ndarray) -> float:
    """Compute circular mean of phases.
    
    Args:
        phases: Array of phase values in radians
        
    Returns:
        Circular mean in radians
    """
    sin_sum = np.sum(np.sin(phases))
    cos_sum = np.sum(np.cos(phases))
    return np.arctan2(sin_sum, cos_sum)


def circular_variance(phases: np.ndarray) -> float:
    """Compute circular variance of phases.
    
    Args:
        phases: Array of phase values in radians
        
    Returns:
        Circular variance (0 = perfectly aligned, 1 = uniform)
    """
    n = len(phases)
    sin_sum = np.sum(np.sin(phases))
    cos_sum = np.sum(np.cos(phases))
    R = np.sqrt(sin_sum**2 + cos_sum**2) / n
    return 1 - R


def circular_std(phases: np.ndarray) -> float:
    """Compute circular standard deviation.
    
    Args:
        phases: Array of phase values in radians
        
    Returns:
        Circular standard deviation
    """
    var = circular_variance(phases)
    return np.sqrt(-2 * np.log(1 - var)) if var < 1 else np.inf


def rayleigh_test(phases: np.ndarray) -> Tuple[float, float]:
    """Rayleigh test for non-uniformity of circular distribution.
    
    Args:
        phases: Array of phase values in radians
        
    Returns:
        Tuple of (R statistic, p-value)
    """
    n = len(phases)
    sin_sum = np.sum(np.sin(phases))
    cos_sum = np.sum(np.cos(phases))
    R = np.sqrt(sin_sum**2 + cos_sum**2) / n
    
    # Rayleigh test statistic
    Z = n * R**2
    
    # P-value approximation
    p_value = np.exp(-Z) * (1 + (2*Z - Z**2) / (4*n) - 
                           (24*Z - 132*Z**2 + 76*Z**3 - 9*Z**4) / (288*n**2))
    
    return R, p_value


def watson_u2_test(phases1: np.ndarray, phases2: np.ndarray) -> Tuple[float, float]:
    """Watson's U2 test for comparing two circular distributions.
    
    Args:
        phases1: First set of phases
        phases2: Second set of phases
        
    Returns:
        Tuple of (U2 statistic, approximate p-value)
    """
    # Combine and sort
    n1, n2 = len(phases1), len(phases2)
    N = n1 + n2
    
    all_phases = np.concatenate([phases1, phases2])
    labels = np.concatenate([np.ones(n1), np.zeros(n2)])
    
    # Sort by phase
    sorted_idx = np.argsort(all_phases)
    sorted_labels = labels[sorted_idx]
    
    # Compute cumulative differences
    d = np.zeros(N)
    cum1 = 0
    cum2 = 0
    for i in range(N):
        if sorted_labels[i] == 1:
            cum1 += 1
        else:
            cum2 += 1
        d[i] = cum1/n1 - cum2/n2
    
    # U2 statistic
    d_bar = np.mean(d)
    U2 = (n1 * n2 / N**2) * np.sum((d - d_bar)**2)
    
    # Approximate p-value (critical values from tables)
    # U2 > 0.187 is significant at p < 0.05
    if U2 > 0.268:
        p_value = 0.01
    elif U2 > 0.187:
        p_value = 0.05
    elif U2 > 0.152:
        p_value = 0.10
    else:
        p_value = 0.50
    
    return U2, p_value


def circular_correlation(phases1: np.ndarray, phases2: np.ndarray) -> float:
    """Compute circular-circular correlation.
    
    Args:
        phases1: First set of phases
        phases2: Second set of phases
        
    Returns:
        Circular correlation coefficient
    """
    sin1 = np.sin(phases1 - circular_mean(phases1))
    sin2 = np.sin(phases2 - circular_mean(phases2))
    
    numerator = np.sum(sin1 * sin2)
    denominator = np.sqrt(np.sum(sin1**2) * np.sum(sin2**2))
    
    if denominator == 0:
        return 0.0
    
    return numerator / denominator


print("Circular statistics functions ready")

## 4. Phase Extraction Methods

In [ ]:
def extract_phase_hilbert(activation: np.ndarray) -> np.ndarray:
    """Extract phase using Hilbert transform.
    
    The Hilbert transform creates an analytic signal from which
    we can extract instantaneous phase.
    
    Args:
        activation: Activation vector
        
    Returns:
        Phase vector (same shape as input)
    """
    # Apply Hilbert transform
    analytic_signal = hilbert(activation)
    
    # Extract phase
    phase = np.angle(analytic_signal)
    
    return phase


def extract_phase_pca(activation: np.ndarray, n_components: int = 2) -> np.ndarray:
    """Extract phase using PCA projection to 2D and computing angle.
    
    Args:
        activation: Activation vector
        n_components: Number of PCA components (2 for phase extraction)
        
    Returns:
        Single phase value
    """
    # This method works on multiple activations
    # For single activation, just return angle of first two components
    if len(activation.shape) == 1:
        return np.arctan2(activation[1], activation[0])
    else:
        from sklearn.decomposition import PCA
        pca = PCA(n_components=n_components)
        projected = pca.fit_transform(activation)
        return np.arctan2(projected[:, 1], projected[:, 0])


def extract_phase_fourier(activation: np.ndarray, n_components: int = 10) -> np.ndarray:
    """Extract phase from dominant Fourier components.
    
    Args:
        activation: Activation vector
        n_components: Number of Fourier components to use
        
    Returns:
        Array of phases for dominant frequency components
    """
    fft = np.fft.fft(activation)
    
    # Get phases of n_components largest magnitude components
    magnitudes = np.abs(fft)
    top_indices = np.argsort(magnitudes)[-n_components:]
    
    phases = np.angle(fft[top_indices])
    
    return phases


print("Phase extraction functions ready")

## 5. Prompt Sets for Phase Testing

In [ ]:
# Compatible AQ pairs (should show phase alignment)
COMPATIBLE_PAIRS = {
    "threat_flee": {
        "prompts": [
            "A dangerous fire threatens you and you must escape. You should",
            "A predator is hunting you and you need to flee. You should",
            "The building is collapsing and you must run away. You should",
            "A venomous snake approaches and you should retreat. You should",
            "The flood is rising and you need to evacuate. You should",
        ] * 10,  # Repeat to get enough samples
        "components": ["threat", "flee"]
    },
    "urgency_act": {
        "prompts": [
            "The deadline is now and you must act immediately. You should",
            "Time is running out and you need to move now. You should",
            "This is your last chance and you must take action. You should",
            "The moment is here and you need to respond instantly. You should",
            "There's no time left and you must do something now. You should",
        ] * 10,
        "components": ["urgency", "action"]
    },
    "proximity_reach": {
        "prompts": [
            "The goal is within arm's reach and you can grab it. You should",
            "Safety is just steps away and you can get there. You should",
            "The prize is right beside you and you can take it. You should",
            "Help is nearby and you can access it easily. You should",
            "The exit is close and you can reach it quickly. You should",
        ] * 10,
        "components": ["proximity", "reach"]
    }
}

# Incompatible AQ pairs (should show phase opposition)
INCOMPATIBLE_PAIRS = {
    "flee_approach": {
        "prompts": [
            "You must run away but also move closer. You should",
            "Escape is necessary but you need to approach. You should",
            "Flee immediately but also advance forward. You should",
            "Retreat quickly but also move toward it. You should",
            "Get away fast but also get closer. You should",
        ] * 10,
        "components": ["flee", "approach"]
    },
    "urgent_wait": {
        "prompts": [
            "Act immediately but also wait patiently. You should",
            "This is urgent but you must be patient. You should",
            "Move now but also stay still. You should",
            "Time is critical but delay is needed. You should",
            "Rush immediately but also hold back. You should",
        ] * 10,
        "components": ["urgent", "wait"]
    },
    "threat_safe": {
        "prompts": [
            "Danger is present but everything is safe. You should",
            "The threat is real but there's no risk. You should",
            "It's hazardous but completely secure. You should",
            "A menace approaches but all is well. You should",
            "Peril exists but safety is assured. You should",
        ] * 10,
        "components": ["threat", "safe"]
    }
}

# Random/control pairs (should show random phase distribution)
RANDOM_PAIRS = {
    "random_1": {
        "prompts": [
            "The weather is nice and books are interesting. You should",
            "Mathematics is useful and music is pleasant. You should",
            "Trees are tall and water is wet. You should",
            "Science explains things and art expresses them. You should",
            "History teaches us and geography shows us. You should",
        ] * 10,
        "components": ["random", "random"]
    }
}

print(f"Compatible pairs: {list(COMPATIBLE_PAIRS.keys())}")
print(f"Incompatible pairs: {list(INCOMPATIBLE_PAIRS.keys())}")
print(f"Random pairs: {list(RANDOM_PAIRS.keys())}")

## 6. Model Loading and Activation Extraction

In [ ]:
print(f"Loading {config.model_name}...")
tokenizer = AutoTokenizer.from_pretrained(config.model_path)
model = AutoModelForCausalLM.from_pretrained(
    config.model_path,
    output_hidden_states=True,
    torch_dtype=torch.float16 if DEVICE == "cuda" else torch.float32
)
model = model.to(DEVICE)
model.eval()

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"Model loaded: {sum(p.numel() for p in model.parameters()) / 1e6:.1f}M parameters")

In [ ]:
def get_activation(prompt: str, model: nn.Module, tokenizer: AutoTokenizer, 
                   layers: List[int]) -> Dict[int, np.ndarray]:
    """Get last token activation at specified layers.
    
    Args:
        prompt: Input text
        model: The model
        tokenizer: The tokenizer
        layers: List of layer indices
        
    Returns:
        Dict mapping layer index to activation vector
    """
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(DEVICE)
    
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    
    activations = {}
    for layer_idx in layers:
        h = outputs.hidden_states[layer_idx][0, -1, :].cpu().float().numpy()
        activations[layer_idx] = h
    
    return activations


print("Activation extraction ready")

## 7. Run Phase Analysis

In [ ]:
def analyze_phase_alignment(prompts: List[str], model, tokenizer, 
                            layers: List[int]) -> Dict[str, Any]:
    """Analyze phase alignment across prompts.
    
    Args:
        prompts: List of prompts
        model: The model
        tokenizer: The tokenizer
        layers: Layers to analyze
        
    Returns:
        Dict with phase analysis results
    """
    results = {"layers": {}}
    
    # Get activations for all prompts
    all_activations = {layer: [] for layer in layers}
    
    for prompt in tqdm(prompts[:config.samples_per_category], desc="Getting activations"):
        acts = get_activation(prompt, model, tokenizer, layers)
        for layer in layers:
            all_activations[layer].append(acts[layer])
    
    # Analyze phase at each layer
    for layer in layers:
        activations = np.array(all_activations[layer])
        
        # Extract phases using multiple methods
        # Method 1: Hilbert transform (per dimension, then aggregate)
        hilbert_phases = []
        for act in activations:
            phase = extract_phase_hilbert(act)
            # Use mean phase across dimensions
            hilbert_phases.append(circular_mean(phase))
        hilbert_phases = np.array(hilbert_phases)
        
        # Method 2: Fourier dominant component
        fourier_phases = []
        for act in activations:
            phase = extract_phase_fourier(act, n_components=1)
            fourier_phases.append(phase[0])
        fourier_phases = np.array(fourier_phases)
        
        # Compute statistics
        hilbert_var = circular_variance(hilbert_phases)
        hilbert_R, hilbert_p = rayleigh_test(hilbert_phases)
        
        fourier_var = circular_variance(fourier_phases)
        fourier_R, fourier_p = rayleigh_test(fourier_phases)
        
        results["layers"][layer] = {
            "hilbert": {
                "circular_variance": float(hilbert_var),
                "rayleigh_R": float(hilbert_R),
                "rayleigh_p": float(hilbert_p),
                "mean_phase": float(circular_mean(hilbert_phases)),
                "circular_std": float(circular_std(hilbert_phases))
            },
            "fourier": {
                "circular_variance": float(fourier_var),
                "rayleigh_R": float(fourier_R),
                "rayleigh_p": float(fourier_p),
                "mean_phase": float(circular_mean(fourier_phases)),
                "circular_std": float(circular_std(fourier_phases))
            },
            "phases_hilbert": hilbert_phases.tolist(),
            "phases_fourier": fourier_phases.tolist()
        }
    
    return results


print("Phase analysis function ready")

In [ ]:
# Run analysis for all prompt categories
print("Analyzing phase relationships...")
print("=" * 60)

RESULTS = {
    "compatible": {},
    "incompatible": {},
    "random": {}
}

# Compatible pairs
print("\nCompatible pairs (expecting LOW phase variance, significant Rayleigh):")
for name, data in COMPATIBLE_PAIRS.items():
    print(f"  Analyzing {name}...")
    RESULTS["compatible"][name] = analyze_phase_alignment(
        data["prompts"], model, tokenizer, config.layers_to_probe
    )

# Incompatible pairs
print("\nIncompatible pairs (expecting phase opposition):")
for name, data in INCOMPATIBLE_PAIRS.items():
    print(f"  Analyzing {name}...")
    RESULTS["incompatible"][name] = analyze_phase_alignment(
        data["prompts"], model, tokenizer, config.layers_to_probe
    )

# Random pairs
print("\nRandom pairs (expecting HIGH phase variance, non-significant Rayleigh):")
for name, data in RANDOM_PAIRS.items():
    print(f"  Analyzing {name}...")
    RESULTS["random"][name] = analyze_phase_alignment(
        data["prompts"], model, tokenizer, config.layers_to_probe
    )

print("\nAnalysis complete.")

## 8. Results Summary

In [ ]:
print("\n" + "=" * 70)
print("PHASE RELATIONSHIP ANALYSIS SUMMARY")
print("=" * 70)

# Find best layer (lowest variance for compatible pairs)
best_layer = None
best_variance = float('inf')

for layer in config.layers_to_probe:
    variances = []
    for name, results in RESULTS["compatible"].items():
        variances.append(results["layers"][layer]["hilbert"]["circular_variance"])
    mean_var = np.mean(variances)
    if mean_var < best_variance:
        best_variance = mean_var
        best_layer = layer

print(f"\nBest layer for phase alignment: {best_layer}")

# Summary table
print(f"\nResults at Layer {best_layer}:")
print("-" * 80)
print(f"{'Category':<20} {'Type':<15} {'Circ.Var':<12} {'Rayleigh R':<12} {'p-value':<12}")
print("-" * 80)

# Compatible
compatible_sig_count = 0
for name, results in RESULTS["compatible"].items():
    layer_data = results["layers"][best_layer]["hilbert"]
    sig = "***" if layer_data["rayleigh_p"] < 0.001 else "**" if layer_data["rayleigh_p"] < 0.01 else "*" if layer_data["rayleigh_p"] < 0.05 else ""
    if layer_data["rayleigh_p"] < 0.05:
        compatible_sig_count += 1
    print(f"{name:<20} {'Compatible':<15} {layer_data['circular_variance']:<12.4f} {layer_data['rayleigh_R']:<12.4f} {layer_data['rayleigh_p']:<10.4f} {sig}")

# Incompatible
print()
for name, results in RESULTS["incompatible"].items():
    layer_data = results["layers"][best_layer]["hilbert"]
    sig = "***" if layer_data["rayleigh_p"] < 0.001 else "**" if layer_data["rayleigh_p"] < 0.01 else "*" if layer_data["rayleigh_p"] < 0.05 else ""
    print(f"{name:<20} {'Incompatible':<15} {layer_data['circular_variance']:<12.4f} {layer_data['rayleigh_R']:<12.4f} {layer_data['rayleigh_p']:<10.4f} {sig}")

# Random
print()
for name, results in RESULTS["random"].items():
    layer_data = results["layers"][best_layer]["hilbert"]
    sig = "***" if layer_data["rayleigh_p"] < 0.001 else "**" if layer_data["rayleigh_p"] < 0.01 else "*" if layer_data["rayleigh_p"] < 0.05 else ""
    print(f"{name:<20} {'Random':<15} {layer_data['circular_variance']:<12.4f} {layer_data['rayleigh_R']:<12.4f} {layer_data['rayleigh_p']:<10.4f} {sig}")

print("-" * 80)
print("\n*** p < 0.001, ** p < 0.01, * p < 0.05")

## 9. Visualization

In [ ]:
# Polar plots of phase distributions
fig, axes = plt.subplots(2, 3, figsize=(15, 10), subplot_kw={'projection': 'polar'})

# Plot compatible pairs
for idx, (name, results) in enumerate(RESULTS["compatible"].items()):
    if idx >= 3:
        break
    ax = axes[0, idx]
    phases = np.array(results["layers"][best_layer]["phases_hilbert"])
    ax.hist(phases, bins=36, density=True, alpha=0.7, color='green')
    ax.set_title(f"Compatible: {name}\nVar={results['layers'][best_layer]['hilbert']['circular_variance']:.3f}")

# Plot incompatible pairs
for idx, (name, results) in enumerate(RESULTS["incompatible"].items()):
    if idx >= 3:
        break
    ax = axes[1, idx]
    phases = np.array(results["layers"][best_layer]["phases_hilbert"])
    ax.hist(phases, bins=36, density=True, alpha=0.7, color='red')
    ax.set_title(f"Incompatible: {name}\nVar={results['layers'][best_layer]['hilbert']['circular_variance']:.3f}")

plt.suptitle(f"035G: Phase Distributions at Layer {best_layer}", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig("035G_phase_distributions.png", dpi=150, bbox_inches='tight')
plt.show()
print("Saved: 035G_phase_distributions.png")

In [ ]:
# Phase variance across layers
fig, ax = plt.subplots(figsize=(12, 6))

# Compatible
for name, results in RESULTS["compatible"].items():
    variances = [results["layers"][l]["hilbert"]["circular_variance"] for l in config.layers_to_probe]
    ax.plot(config.layers_to_probe, variances, 'o-', label=f"Compatible: {name}", color='green', alpha=0.7)

# Incompatible
for name, results in RESULTS["incompatible"].items():
    variances = [results["layers"][l]["hilbert"]["circular_variance"] for l in config.layers_to_probe]
    ax.plot(config.layers_to_probe, variances, 's--', label=f"Incompatible: {name}", color='red', alpha=0.7)

# Random
for name, results in RESULTS["random"].items():
    variances = [results["layers"][l]["hilbert"]["circular_variance"] for l in config.layers_to_probe]
    ax.plot(config.layers_to_probe, variances, '^:', label=f"Random: {name}", color='gray', alpha=0.7)

ax.set_xlabel("Layer")
ax.set_ylabel("Circular Variance (lower = more aligned)")
ax.set_title("Phase Alignment by Layer and Category Type")
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig("035G_phase_variance_by_layer.png", dpi=150, bbox_inches='tight')
plt.show()
print("Saved: 035G_phase_variance_by_layer.png")

## 10. Conclusions

In [ ]:
print("\n" + "=" * 70)
print("EXPERIMENT 035G: PHASE RELATIONSHIPS CONCLUSIONS")
print("=" * 70)

print(f"\nExperiment Configuration:")
print(f"  Model: {config.model_name}")
print(f"  Samples per category: {config.samples_per_category}")
print(f"  Layers analyzed: {config.layers_to_probe}")

# Compute success criteria
n_compatible = len(RESULTS["compatible"])
n_compatible_sig = compatible_sig_count

# Compare variances
compatible_vars = []
incompatible_vars = []
random_vars = []

for results in RESULTS["compatible"].values():
    compatible_vars.append(results["layers"][best_layer]["hilbert"]["circular_variance"])
for results in RESULTS["incompatible"].values():
    incompatible_vars.append(results["layers"][best_layer]["hilbert"]["circular_variance"])
for results in RESULTS["random"].values():
    random_vars.append(results["layers"][best_layer]["hilbert"]["circular_variance"])

mean_compatible_var = np.mean(compatible_vars)
mean_incompatible_var = np.mean(incompatible_vars)
mean_random_var = np.mean(random_vars)

print(f"\nPhase Variance Summary (Layer {best_layer}):")
print(f"  Compatible pairs:   {mean_compatible_var:.4f}")
print(f"  Incompatible pairs: {mean_incompatible_var:.4f}")
print(f"  Random pairs:       {mean_random_var:.4f}")

print(f"\nHypothesis Testing:")
print(f"  1. Compatible < Random variance: {mean_compatible_var < mean_random_var}")
print(f"  2. Rayleigh significant for compatible: {n_compatible_sig}/{n_compatible}")

# Overall conclusion
success = (mean_compatible_var < mean_random_var) and (n_compatible_sig >= n_compatible // 2)

print(f"\n" + "=" * 70)
if success:
    print("CONCLUSION: Evidence SUPPORTS AQ phase alignment hypothesis.")
    print("Compatible AQ pairs show lower phase variance than random pairs.")
else:
    print("CONCLUSION: Evidence does NOT clearly support AQ phase hypothesis.")
    print("Phase structure may require different extraction methods or more data.")
print("=" * 70)

In [ ]:
# Save results
def make_serializable(obj):
    """Convert numpy types for JSON."""
    if isinstance(obj, dict):
        return {str(k): make_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [make_serializable(v) for v in obj]
    elif isinstance(obj, (np.integer, np.floating)):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        return obj

results_output = {
    "config": {
        "model": config.model_name,
        "samples_per_category": config.samples_per_category,
        "layers": config.layers_to_probe
    },
    "results": make_serializable(RESULTS),
    "summary": {
        "best_layer": best_layer,
        "mean_compatible_variance": float(mean_compatible_var),
        "mean_incompatible_variance": float(mean_incompatible_var),
        "mean_random_variance": float(mean_random_var),
        "compatible_rayleigh_significant": n_compatible_sig,
        "conclusion": "SUPPORTS" if success else "DOES NOT SUPPORT"
    }
}

with open("035G_results.json", "w") as f:
    json.dump(results_output, f, indent=2)

print("Results saved to 035G_results.json")